In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras

In [13]:
sample = tf.constant(list(range(1000*224*224*3)), dtype=tf.float32, shape=(1000, 224, 224, 3))
sample_label = tf.constant(list(range(1000)), dtype=tf.int64)

define inception module !

In [17]:
def inception_module(x, fileters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool):
  conv_1x1 = keras.layers.Conv2D(fileters_1x1, 1, padding = 'same', activation = 'relu', )(x)

  pre_conv_3x3 = keras.layers.Conv2D(filters_3x3_reduce, 1, padding = 'same', activation = 'relu', )(x)
  conv_3x3 = keras.layers.Conv2D(filters_3x3, 3, padding = 'same', activation = 'relu', )(pre_conv_3x3)

  pre_conv_5x5 = keras.layers.Conv2D(filters_5x5_reduce, 1, padding = 'same', activation = 'relu', )(x)
  conv_5x5 = keras.layers.Conv2D(filters_5x5, 5, padding = 'same', activation = 'relu', )(pre_conv_5x5)

  pre_pool = keras.layers.MaxPooling2D(pool_size = (3, 3), strides = 1, padding = 'same')(x)
  pooling = keras.layers.Conv2D(filters_pool, 1, padding = 'same', activation = 'relu', )(pre_pool)

  output = keras.layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pooling], axis = 3)

  return output

In [18]:
input_layer = keras.layers.Input(shape = (224, 224, 3))
x = keras.layers.Conv2D(64, 7, strides = 2, padding='same', activation='relu')(input_layer)
x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides = 2, padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(64, 1, strides = 1, padding='same', activation='relu')(x)
x = keras.layers.Conv2D(192, 3, strides = 1, padding='same', activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides = 2, padding='same')(x)

x = inception_module(x, 64, 96, 128, 16, 32, 32)
x = inception_module(x, 128, 128, 192, 32, 96, 64)
x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides = 2, padding='same')(x)

x = inception_module(x, 192, 96, 208, 16, 48, 64)
x = inception_module(x, 160, 112, 224, 24, 64, 64)
x = inception_module(x, 128, 128, 256, 24, 64, 64)
x = inception_module(x, 112, 144, 288, 32, 64, 64)
x = inception_module(x, 256, 160, 320, 32, 128, 128)
x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides = 2, padding='same')(x)

x = inception_module(x, 256, 160, 320, 32, 128, 128)
x = inception_module(x, 384, 192, 384, 48, 128, 128)

x = keras.layers.AveragePooling2D(pool_size = (7, 7), strides = 1, padding = 'valid')(x)
x = keras.layers.Dropout(0.4)(x)
output = keras.layers.Dense(1000)(x)

In [19]:
model = keras.models.Model(input_layer, output, name='googlenet')

In [20]:
model.summary()

Model: "googlenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_72 (Conv2D)             (None, 112, 112, 64  9472        ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_24 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_72[0][0]']              
 )                                                                                        

In [22]:
import math

epochs = 1
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = keras.optimizers.SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = keras.callbacks.LearningRateScheduler(decay, verbose=1)

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits = True), optimizer=sgd, metrics=['accuracy'])

model.fit(sample, sample_label, epochs=epochs, callbacks=[lr_sc])


Epoch 1: LearningRateScheduler setting learning rate to 0.01.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


32/32 [==============================] - 8s 170ms/step - loss: 6.9306 - accuracy: 5.3446e-04 - lr: 0.0100
